# Setup

## Verify we're in the Conda environment

In [ ]:
import sys

print(sys.executable)

## Import python packages

In [11]:
import os
import sys
import json
from PIL import Image
import base64
import io
from dotenv import load_dotenv
import requests
import pprint
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import subprocess
import jupyter_black

# Activate the jupyter_black extension, which reformats code cells with black
# https://github.com/n8henrie/jupyter-black
jupyter_black.load()

In [ ]:
import hashlib


def calculate_hash_seed(sentence, num_try):
    # Encode the sentence to bytes
    sentence_bytes = sentence.encode("utf-8")

    # Create a hash object
    hash_object = hashlib.sha256()

    # Update the hash object with the bytes
    hash_object.update(sentence_bytes)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_object.hexdigest()

    # Convert the hexadecimal hash to a decimal number
    hash_decimal = int(hash_hex, 16)

    # Make it more random by multiplying by the number of tries
    hash_decimal *= num_try * 100

    # Limit the hash value to the range of a 32-bit signed integer
    seed = hash_decimal % (2**31 - 1)

    return seed


# Example usage
sentence = "This is a sample sentence."
seed_value = calculate_hash_seed(sentence, 1)
print(f"The hash of the sentence as a seed is: {seed_value}")

In [ ]:
import re


def filter_text(text):
    # Only keep the first line of the answer
    text = text.split("\n")[0]

    # Remove quotes from the answer. Both single and double quotes are removed.
    text = text.replace('"', "").replace("'", "")

    # Remove leading and trailing whitespaces
    text = text.strip()

    # Use regex to remove special characters
    text = re.sub(r"[^\x00-\x7F]+", "", text)

    # Remove backslash from the answer.
    text = text.replace("\\", "")

    # Remove [end of text]
    text = text.replace("[end of text]", "")

    return text


# Example usage
text = "Byzantine\u5171\u8bc6 ('Byzantine' \"consensus\")\n next line must be removed [end of text]"
filtered_text = filter_text(text)
print(filtered_text)

In [14]:
# Define where the llama-cli is located, relative to this notebook
LLAMA_CLI_PATH = "../../../ggerganov_llama_615212.cpp/build/bin/llama-cli"
# LLAMA_CLI_PATH = "../../../ggerganov_llama_latest.cpp/build/bin/llama-cli"

# Select a model to use
MODEL = "../../../llama_cpp_canister/models/Qwen/Qwen2.5-0.5B-Instruct-GGUF/qwen2.5-0.5b-instruct-q8_0.gguf"
# MODEL = "../../../llama_cpp_canister/models/tensorblock/SmolLM2-135M-Instruct-GGUF/SmolLM2-135M-Instruct-Q8_0.gguf"
# MODEL = (
#     "../../../llama_cpp_canister/models/tensorblock/SmolLM2-135M-Instruct-GGUF/SmolLM2-135M-Instruct-Q4_K_M.gguf"
# )
# MODEL = "../../../llama_cpp_canister/models/unsloth/DeepSeek-R1-Distill-Qwen-1.5B-GGUF/DeepSeek-R1-Distill-Qwen-1.5B-Q2_K.gguf"
# MODEL = "../../../llama_cpp_canister/models/unsloth/DeepSeek-R1-Distill-Qwen-7B-GGUF/DeepSeek-R1-Distill-Qwen-7B-Q2_K.gguf"

print_command = True


def run_llama_cpp(prompt, num_tokens, seed, temp, top_k, top_p, min_p):
    command = [
        LLAMA_CLI_PATH,
        "-m",
        MODEL,
        "--no-warmup",  # needed when running from CLI. Is default for llama_cpp_canister
        "-no-cnv",  # needed when running from CLI. Is default for llama_cpp_canister
        "--simple-io",
        "--no-display-prompt",  # only return the generated text, without special characters
        # "-sp", # output special tokens
        "-n",
        f"{num_tokens}",
        "--seed",
        f"{seed}",
        "--temp",
        f"{temp}",
        "--top-k",
        f"{top_k}",
        "--top-p",
        f"{top_p}",
        "--min-p",
        f"{min_p}",
        "-p",
        prompt,
    ]

    # print this only once !
    global print_command
    if print_command:
        print_command = False
        # Print the command on a single line for terminal use, preserving \n
        print(
            "\nCommand:\n",
            f"{LLAMA_CLI_PATH} -m {MODEL} --no-warmup -no-cnv --simple-io --no-display-prompt -n {num_tokens} --seed {seed} --temp {temp} -p '{prompt}'".replace(
                "\n", "\\n"
            ),
        )

    # Run the command and capture the output
    result = subprocess.run(
        command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    output = result.stdout
    return output

In [ ]:
import json
import pprint

# Specify the file path
file_path = "1-challenges.json"

# Read the list from the JSON file
with open(file_path, "r") as file:
    challenges = json.load(file)

print(f"Challenges have been read from the file: {file_path}")

pprint.pprint(challenges[:3])

In [16]:
mainer_ids = [
    "4hfoe-6aaaa-aaaaj-qnfwq-cai",
    "4ogfy-iiaaa-aaaaj-qnfxa-cai",
    "4jhdm-fqaaa-aaaaj-qnfxq-cai",
    "6u2ne-wyaaa-aaaaj-qnfya-cai",
    "lkh5o-3yaaa-aaaag-acguq-cai",
    "laxfu-oqaaa-aaaag-ak5kq-cai",
    "ljuoi-yyaaa-aaaag-ak5la-cai",
    "lovi4-vaaaa-aaaag-ak5lq-cai",
    "tsigi-iyaaa-aaaaj-azxea-cai",
    "tvja4-faaaa-aaaaj-azxeq-cai",
    "t4kla-tiaaa-aaaaj-azxfa-cai",
    "t3lnu-6qaaa-aaaaj-azxfq-cai",
]

# Initial seeds
# mainer_seeds = [calculate_hash_seed(mainer_id) for mainer_id in mainer_ids]

In [ ]:
challenges_with_answers = challenges.copy()

mainer_num_tokens = 20

num_tries = 10

for challenge in challenges_with_answers:
    challenge_id = challenge["challenge_id"]
    challenge_topic = challenge["challenge_topic"]
    challenge_question = challenge["challenge_question"]

    # Dynamically add the mainer_answers key to the challenge dictionary
    # (Envisioning that the caller to the mainers will do it like this...)
    challenge["mainer_answers"] = []

    print("-------------------")
    print(
        f"challenge_id: {challenge_id}, topic: {challenge_topic} - challenge_question: {challenge_question}"
    )

    mainer_top_k = 40  # (default: 40, 0 = disabled)
    mainer_top_p = 0.9  # (default: 0.9, 1.0 = disabled)
    mainer_min_p = 0.1  # (default: 0.1, 0.0 = disabled)
    for i, mainer_id in enumerate(mainer_ids):
        for mainer_temp in [
            # 3.0,
            # 2.5,
            # 2.0,
            # 1.75,
            # 1.5,
            # 1.25,
            # 1.0,
            0.7,
            # 0.6,
            # 0.4,
            # 0.2,
            # 0.0,
        ]:
            # for mainer_top_p in [
            #     1.0,
            #     # 0.9, 0.8, 0.6, 0.4, 0.2
            # ]:

            # mainer_prompt = f"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nThis is a question about {challenge_topic}, that can be answered with common knowledge. Give the answer as brief as possible. This is the question: {challenge_question}\n<|im_end|>\n<|im_start|>assistant\nThe answer is:"
            mainer_prompt = f"<|im_start|>user\nThis is a question about {challenge_topic}. Give the answer as brief as possible. This is the question: {challenge_question}\n<|im_end|>\n<|im_start|>assistant\nThe answer is:"
            # mainer_prompt = f"<|im_start|>user\nAnswer this question as brief as possible.\nThis is the question: {challenge_question}\n<|im_end|>\n<|im_start|>assistant\nThe answer is:"

            for mainer_try in range(num_tries):
                # mainer_seed = mainer_seeds[i] * (mainer_try +1)
                if mainer_try < 3:
                    mainer_seed = i * (mainer_try + 1) * 10 + i + mainer_try
                elif mainer_try < 6:
                    mainer_seed = (
                        (i + 1) * (mainer_try + 1) * 100
                        + i * (mainer_try + 1) * 10
                        + i
                        + mainer_try
                    )
                else:
                    mainer_seed = calculate_hash_seed(mainer_id, mainer_try)
                # mainer_seed = -1
                mainer_answer = run_llama_cpp(
                    mainer_prompt,
                    mainer_num_tokens,
                    mainer_seed,
                    mainer_temp,
                    mainer_top_k,
                    mainer_top_p,
                    mainer_min_p,
                )

                mainer_answer = filter_text(mainer_answer)
                challenge["mainer_answers"].append(
                    {
                        "mainer_id": mainer_id,
                        "mainer_try": mainer_try,
                        "mainer_seed": mainer_seed,
                        "mainer_answer": mainer_answer,
                        "mainer_temp": mainer_temp,
                        "mainer_top_p": mainer_top_p,
                    }
                )

                print(
                    f"mainer_id: {mainer_id}, mainer_try: {mainer_try}, mainer_seed: {mainer_seed}, mainer_temp: {mainer_temp}, mainer_top_p: {mainer_top_p}, mainer_answer: {mainer_answer}"
                )

                # calculate seed for next iteration
                # mainer_seeds[i] = calculate_hash_seed(
                #     str(mainer_seed) + mainer_id + mainer_answer
                # )
                # if mainer_seeds[i] == mainer_seed:
                #     mainer_seeds[i] = calculate_hash_seed(
                #         str(mainer_seed)
                #         + mainer_id
                #         + mainer_answer
                #         + str(mainer_seed)
                #         + mainer_id
                #         + mainer_answer
                #     )

                # quick check to see data is stored correctly
                # pprint.pprint(challenges_with_answers[:3])

                # On a regular basis, save it, so we can monitor progress
                # Specify the file path
                file_path = "2-mainer.json"

                # Write the list to a JSON file
                with open(file_path, "w") as file:
                    json.dump(challenges_with_answers, file, indent=4)